# License

    Jupyter notebook for accessing CUDA
    Copyright (C) 2018 Andre.Brodtkorb@ifi.uio.no, changed in October by André Brodtkorb

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [1]:
#Lets have matplotlib "inline"
%matplotlib inline

#Import packages we need
import numpy as np
import pycuda.compiler as cuda_compiler
from pycuda.gpuarray import GPUArray
import pycuda.driver as cuda_driver

from matplotlib import pyplot as plt

import IPythonMagic

In [2]:
import pytest
from ipytest import run_tests, run_pytest, clean_tests

In [3]:
from Timer import Timer
import logging

In [4]:
%setup_logging
%cuda_context_handler context

Python version 3.6.6 (default, Sep 12 2018, 18:26:19) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]
Registering context in user workspace
Creating context
PyCUDA version 2018.1.1
CUDA version (9, 1, 0)
Driver version 10000
Using 'Tesla K80' GPU
 => compute capability: (3, 7)
 => memory: 10455 / 11441 MB available
Created context handle <31985072>
Using CUDA cache dir /home/ubuntu/jupyter_notebooks/Michele_Pellegrino/MilanoGPU2018/notebooks/cuda_cache


In [5]:
cuda_kernel = """
__global__ void matrixVectorKernel(float* c, float* A, float* b, int a_rows, int a_cols) {
    // Horizontal threads better than vertical threads
    unsigned int j = blockIdx.x*blockDim.x + threadIdx.x;
    
    //Out of bounds check
    if (j > a_rows) {
        return;
    }
    
    //Compute inner product of row of A with column of B
    float sum = 0.0f;
    for (int i=0; i<a_cols; ++i) {
        unsigned int k = j*a_cols + i;
        sum += A[k] * b[i];
    }
    
    //Write to global memory
    c[j] = sum;
}
"""
module = cuda_compiler.SourceModule(cuda_kernel)
kernel = module.get_function("matrixVectorKernel");

In [6]:
def gpuMatrixVector(a, b):
    
    context.synchronize()
    
    #Upload data to the device
    #NOTE: We need to make sure that a=(a_rows, a_columns)
    # and that b=(a_colmuns, 1) (column vector)
    # and that c=(a_rows, 1)
    c = np.empty((a.shape[0], 1), dtype=np.float32)
    
    with Timer("Allocation") as t:
        a_g = GPUArray(a.shape, np.float32)
        b_g = GPUArray(b.shape, np.float32)
        c_g = GPUArray(c.shape, np.float32)
        context.synchronize()
        
    with Timer("Upload") as t:
        a_g.set(a)
        b_g.set(b)
        context.synchronize()
        
    #Allocate output data
    #NOTE: We need to change this so that the grid*block is x = 1, y = number of rows in A
    block_size = (128, 1, 1) #These need to be [x, y, z]
    grid_size = (int(np.ceil(c.shape[0] / 128)), 1, 1)
    print("Block size is " + str(block_size))
    print("Grid size is " + str(grid_size))
    #Execute program on device
    
    with Timer("Execution") as t:
        kernel(c_g, a_g, b_g, np.int32(a.shape[0]), np.int32(a.shape[1]), block=block_size, grid=grid_size)
        context.synchronize()
    
    #Copy data from device to host
    c_g.get(c)
    return c 

    # Make other tests for the execution!

In [7]:
#Size of our test
test_size = (10000, 10000)

#Create test input / output data
a = np.random.random(test_size).astype(np.float32)
b = np.random.random((test_size[1], 1)).astype(np.float32)
c = gpuMatrixVector(a, b)

# fig = plt.figure()
# plt.subplot(1,3,1)
# plt.imshow(a)
# plt.subplot(1,3,2)
# plt.imshow(b)
# plt.subplot(1,3,3)
# plt.imshow(c)
# fig.show()

Allocation: 1.788378 ms
Upload: 55.014372 ms
Execution: 34.315109 ms


Block size is (128, 1, 1)
Grid size is (79, 1, 1)


In [8]:
#Compute reference using Numpy
c_ref = np.dot(a, b)

#Sum of absolute differences
sad = np.sum(np.abs(c - c_ref))

#Print result
# print("C   = ", c)
# print("Ref = ", c_ref)
print("Sad = %.30f" % sad)
print("Per element error: " + str(sad / test_size[1]))

Sad = 30.949707031250000000000000000000
Per element error: 0.003094970703125


In [9]:
clean_tests()

def test_gpuMatrixVector():
    # Test matrix 1x1
    a = np.ones((1,1), dtype=np.float32)
    b = 2*np.ones((1,1), dtype=np.float32)
    c = gpuMatrixVector(a, b)
    assert c == pytest.approx(2.0)
    # Test inner product 1x1
    a = np.ones((1,2), dtype=np.float32)
    b = 2*np.ones((2,1), dtype=np.float32)
    c = gpuMatrixVector(a, b)
    assert c == pytest.approx(4.0)
    # Test general matrix
    test_size = (4,3)
    a = np.random.random(test_size).astype(np.float32)
    b = np.random.random((test_size[1], 1)).astype(np.float32)
    c = gpuMatrixVector(a, b)
    assert c == pytest.approx(np.dot(a,b))

In [10]:
run_pytest(filename='MatrixVectorTesting.ipynb', pytest_options=['-vvv'])

============================= test session starts ==============================
platform linux -- Python 3.6.6, pytest-3.8.2, py-1.6.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /home/ubuntu/jupyter_notebooks/Michele_Pellegrino/MilanoGPU2018/notebooks, inifile:
collecting ... collected 1 item

MatrixVectorTesting.py::test_gpuMatrixVector <- <ipython-input-9-f218d36f4ccb> 

Allocation: 0.607967 ms
Upload: 0.397682 ms
Execution: 0.153780 ms
Allocation: 0.591278 ms
Upload: 0.412703 ms
Execution: 0.571728 ms
Allocation: 0.473261 ms
Upload: 0.475407 ms
Execution: 0.191212 ms


PASSED [100%]

=========================== 1 passed in 0.04 seconds ===========================


0

In [11]:
def timing_gpuMatrixVector(dim_char):
    # Test matrix
    a = np.ones((dim_char,dim_char), dtype=np.float32)
    b = np.ones((dim_char,1), dtype=np.float32)
    c = gpuMatrixVector(a, b)
    assert c == pytest.approx(np.dot(a,b))

In [12]:
for k in [1, 10, 100, 1000, 10000]:
    with Timer("My Timer Tag", logging.INFO) as t:
        timing_gpuMatrixVector(k)
    print("It took " + str(t.secs) + " seconds")


Allocation: 0.409365 ms
Upload: 1.056194 ms
Execution: 0.105858 ms
My Timer Tag: 5.189896 ms
Allocation: 0.336647 ms
Upload: 0.486612 ms
Execution: 0.087976 ms
My Timer Tag: 3.832340 ms
Allocation: 0.449419 ms
Upload: 0.189781 ms
Execution: 0.464678 ms
My Timer Tag: 11.790991 ms
Allocation: 2.231121 ms
Upload: 2.360106 ms
Execution: 0.811338 ms
My Timer Tag: 21.872520 ms


Block size is (128, 1, 1)
Grid size is (1, 1, 1)
It took 0.0051898956298828125 seconds
Block size is (128, 1, 1)
Grid size is (1, 1, 1)
It took 0.0038323402404785156 seconds
Block size is (128, 1, 1)
Grid size is (1, 1, 1)
It took 0.011790990829467773 seconds
Block size is (128, 1, 1)
Grid size is (8, 1, 1)
It took 0.021872520446777344 seconds


Allocation: 1.113653 ms
Upload: 49.981117 ms
Execution: 29.795885 ms
My Timer Tag: 454.941273 ms


Block size is (128, 1, 1)
Grid size is (79, 1, 1)
It took 0.4549412727355957 seconds
